In [2]:
import json
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer

data_path = r'C:\Users\Subways-Sun\OneDrive\Documents\GitHub\sodium-ion-batteries\data_annotated\search_20241106-223705_sodium+ion+battery+anode-sodium+ion+battery+cathode-sodium+ion+battery+electrode_annotated_rephrased_once.json'
with open(data_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

texts = []
labels_raw = []

for i in data:
    texts.append(i["text"])
    labels_raw.append(i["label_int"])

labels = []
for i in labels_raw:
    if i == 0:
        labels.append(1)
    elif i == 1:
        labels.append(0)

train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts,
    labels,
    test_size=0.2,
    random_state=42
)

train_dataset = Dataset.from_dict({
    'text': train_texts,
    'labels': train_labels
})

val_dataset = Dataset.from_dict({
    'text': val_texts,
    'labels': val_labels
})

raw_datasets = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset
})
print(len(texts))

200


In [2]:
print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 160
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 40
    })
})


In [3]:
model_name = "batterydata/batterybert-cased-abstract"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

In [5]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, torch_dtype="auto")

In [6]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [7]:
import evaluate

metric = evaluate.load("accuracy")

In [8]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [9]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer",
                                  eval_strategy="epoch",
                                  num_train_epochs=10)

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)


In [11]:
trainer.train()

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 0.28721633553504944, 'eval_accuracy': 0.875, 'eval_runtime': 0.3436, 'eval_samples_per_second': 116.407, 'eval_steps_per_second': 14.551, 'epoch': 1.0}


  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 0.5495161414146423, 'eval_accuracy': 0.9, 'eval_runtime': 0.3414, 'eval_samples_per_second': 117.177, 'eval_steps_per_second': 14.647, 'epoch': 2.0}


...

  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 0.13318948447704315, 'eval_accuracy': 0.975, 'eval_runtime': 0.3378, 'eval_samples_per_second': 118.428, 'eval_steps_per_second': 14.803, 'epoch': 9.0}


  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 0.12981636822223663, 'eval_accuracy': 0.975, 'eval_runtime': 0.3299, 'eval_samples_per_second': 121.234, 'eval_steps_per_second': 15.154, 'epoch': 10.0}
{'train_runtime': 44.6343, 'train_samples_per_second': 35.847, 'train_steps_per_second': 4.481, 'train_loss': 0.19759794235229491, 'epoch': 10.0}


TrainOutput(global_step=200, training_loss=0.19759794235229491, metrics={'train_runtime': 44.6343, 'train_samples_per_second': 35.847, 'train_steps_per_second': 4.481, 'total_flos': 420977688576000.0, 'train_loss': 0.19759794235229491, 'epoch': 10.0})

In [12]:
trainer.evaluate()

  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 0.12981636822223663,
 'eval_accuracy': 0.975,
 'eval_runtime': 0.6045,
 'eval_samples_per_second': 66.166,
 'eval_steps_per_second': 8.271,
 'epoch': 10.0}

In [13]:
trainer.save_model("batterybert-cased-abstract_finetuned_200")

In [4]:
from transformers import pipeline
classifier = pipeline("text-classification", model="batterybert-cased-abstract_finetuned_200", tokenizer=tokenizer, device=0)

In [15]:
text = r"MXene nanosheets and ordered porous carbons both have their own advantages and disadvantages. Assembling and combining the advantages of the two will be a good choice for battery electrode hosts of active materials. In this work, an electrostatic separation-adsorption strategy is proposed to realize the ordered alternating self-assembly of MXene nanosheets and ordered porous carbon (MPOC), obtaining a unique wall-like porous material with a high conductivity and interconnected porous nanostructure, which strengthens the transfer rate of electrons and ions simultaneously. Meanwhile, the introduction of N-doping from porous carbon into MPOC prolongs the cycle life. When use red phosphorus (RP) as active materials, the MPOC@RP anode exhibited high-capacity output (2454.3 and 2408.1 mAh g-1 in lithium-ion batteries (LIBs) and sodium-ion batteries (SIBs) at 0.1 C) and long cycle life (the decay rates per cycle of 0.028% and 0.036% after 1500 and 1200 cycles at 2 C in LIBs and SIBs respectively). The successful application in RP anodes displays great potential in other electrode materials such as silicon, sulfur, selenium, and so on. Meanwhile, this strategy is also effective to design other composites materials like MXene and carbon nanotubes, MXene and Graphene, and so on."

In [ ]:
# model_out = AutoModelForSequenceClassification.from_pretrained("batterybert-cased-abstract_finetuned_200")
# with torch.no_grad():
#     logits = model_out(text).logits
# predicted_class_id = logits.argmax().item()
# print(predicted_class_id)

In [16]:
classifier(r"MXene nanosheets and ordered porous carbons both have their own advantages and disadvantages. Assembling and combining the advantages of the two will be a good choice for battery electrode hosts of active materials. In this work, an electrostatic separation-adsorption strategy is proposed to realize the ordered alternating self-assembly of MXene nanosheets and ordered porous carbon (MPOC), obtaining a unique wall-like porous material with a high conductivity and interconnected porous nanostructure, which strengthens the transfer rate of electrons and ions simultaneously. Meanwhile, the introduction of N-doping from porous carbon into MPOC prolongs the cycle life. When use red phosphorus (RP) as active materials, the MPOC@RP anode exhibited high-capacity output (2454.3 and 2408.1 mAh g-1 in lithium-ion batteries (LIBs) and sodium-ion batteries (SIBs) at 0.1 C) and long cycle life (the decay rates per cycle of 0.028% and 0.036% after 1500 and 1200 cycles at 2 C in LIBs and SIBs respectively). The successful application in RP anodes displays great potential in other electrode materials such as silicon, sulfur, selenium, and so on. Meanwhile, this strategy is also effective to design other composites materials like MXene and carbon nanotubes, MXene and Graphene, and so on.")

[{'label': 'battery', 'score': 0.9933772683143616}]

In [17]:
classifier(r"Highly crystalline CuFeS2 containing earth-abundant and environmentally friendly elements prepared via a high-temperature synthesis exhibits an excellent electrochemical performance as an anode material in sodium-ion batteries. The initial specific capacity of 460 mAh g-1 increases to 512 mAh g-1 in the 150th cycle and then decreases to a still very high value of 444 mAh g-1 at 0.5 A g-1 in the remaining 550 cycles. Even for a large current density, a pronounced cycling stability is observed. Here, we demonstrate that combining the results of X-ray powder diffraction experiments, pair distribution function analysis, and 23Na NMR and Mössbauer spectroscopy investigations performed at different stages of discharging and charging processes allows elucidation of very complex reaction mechanisms. In the first step after uptake of 1 Na/CuFeS2, nanocrystalline NaCuFeS2 is formed as an intermediate phase, which surprisingly could be recovered during charging. On increasing the Na content, Cu+ is reduced to nanocrystalline Cu, while nanocrystalline Na2S and nanosized elemental Fe are formed in the discharged state. After charging, the main crystalline phase is NaCuFeS2. At the 150th cycle, the mechanisms clearly changed, and in the charged state, nanocrystalline CuxS phases are observed. At later stages of cycling, the mechanisms are altered again: NaF, Cu2S, and Cu7.2S4 appeared in the discharged state, while NaF and Cu5FeS4 are observed in the charged state. In contrast to a typical conversion reaction, nanocrystalline phases play the dominant role, which are responsible for the high reversible capacity and long-term stability.")[0]["label"]

'battery'

In [5]:
val_path = r'C:\Users\Subways-Sun\OneDrive\Documents\GitHub\sodium-ion-batteries\data_annotated\annotated_data_openai.json'
with open(val_path, 'r', encoding='utf-8') as f:
    val_data = json.load(f)

label_bert = []
score_bert = []
for i in range(len(val_data["text"])):
    val_label = classifier(val_data["text"][i][:512])
    if val_label[0]["label"] == "battery":
        label_bert.append(1)
        score_bert.append(val_label[0]["score"])
    elif val_label[0]["label"] == "non-battery":
        label_bert.append(0)
        score_bert.append(val_label[0]["score"])

val_data["label_bert"] = label_bert
val_data["score_bert"] = score_bert

# result_path = r'C:\Users\Subways-Sun\OneDrive\Documents\GitHub\sodium-ion-batteries\data_annotated\annotated_data_openai_bert.json'
# with open(result_path, 'w', encoding='utf-8') as f:
#     json.dump(val_data, f, indent=4, ensure_ascii=False)



You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [6]:
label_openai = val_data["label_openai"]
label_annotated = val_data["label_annotated"]
print(f"No\tAnno\tOpenAI\tBERT\tBERT Score")

openai_tp = 0
openai_tn = 0
openai_fp = 0
openai_fn = 0
bert_tp = 0
bert_tn = 0
bert_fp = 0
bert_fn = 0

for i in range(len(label_bert)):
    if (label_bert[i] + label_openai[i] + label_annotated[i] != 3) and (label_bert[i] + label_openai[i] + label_annotated[i] != 0):
        print(f"{i}\t{label_annotated[i]}\t{label_openai[i]}\t{label_bert[i]}\t{score_bert[i]}")
        if label_bert[i] == 1 and label_annotated[i] == 0:
            bert_fp += 1
        elif label_bert[i] == 0 and label_annotated[i] == 1:
            bert_fn += 1
        if label_openai[i] == 1 and label_annotated[i] == 0:
            openai_fp += 1
        elif label_openai[i] == 0 and label_annotated[i] == 1:
            openai_fn += 1
    if label_bert[i] == 1 and label_annotated[i] == 1:
        bert_tp += 1
    if label_bert[i] == 0 and label_annotated[i] == 0:
        bert_tn += 1
    if label_openai[i] == 1 and label_annotated[i] == 1:
        openai_tp += 1
    if label_openai[i] == 0 and label_annotated[i] == 0:
        openai_tn += 1
        
print(f"OpenAI True Positive: {openai_tp}")
print(f"OpenAI True Negative: {openai_tn}")
print(f"OpenAI False Positive: {openai_fp}")
print(f"OpenAI False Negative: {openai_fn}")
print(f"Bert True Positive: {bert_tp}")
print(f"Bert True Negative: {bert_tn}")
print(f"Bert False Positive: {bert_fp}")
print(f"Bert False Negative: {bert_fn}")

No	Anno	OpenAI	BERT	BERT Score
4	0	1	0	0.998347282409668
5	0	1	0	0.9998418092727661
8	1	0	1	0.9927880167961121
9	0	0	1	0.9926919341087341
11	1	1	0	0.9997076392173767
13	1	1	0	0.9994577765464783
14	0	0	1	0.9927389025688171
17	0	0	1	0.9918510317802429
20	0	0	1	0.9926034808158875
22	1	0	1	0.9819736480712891
25	0	0	1	0.9925766587257385
29	1	1	0	0.9997610449790955
36	0	0	1	0.9929375052452087
44	0	0	1	0.9925138354301453
45	1	1	0	0.9997242093086243
46	0	0	1	0.930476725101471
49	0	0	1	0.99140465259552
51	1	1	0	0.9996803998947144
52	0	0	1	0.9820249080657959
53	0	0	1	0.9929988384246826
58	0	0	1	0.9920388460159302
60	0	0	1	0.981673002243042
62	1	1	0	0.9982491731643677
63	0	1	0	0.9998726844787598
64	0	0	1	0.9906978011131287
66	0	0	1	0.993403971195221
68	1	0	1	0.9861389994621277
75	0	0	1	0.992253839969635
77	1	1	0	0.9992577433586121
88	0	0	1	0.9931451082229614
93	1	1	0	0.9983423948287964
94	0	0	1	0.9918240904808044
95	0	1	0	0.9998968839645386
97	0	0	1	0.9902768135070801
98	1	1	0	0.9991692304611206


In [7]:
val_path = r'C:\Users\Subways-Sun\OneDrive\Documents\GitHub\sodium-ion-batteries\data_annotated\annotated_data_openai_1.json'
with open(val_path, 'r', encoding='utf-8') as f:
    val_data = json.load(f)

label_bert = []
score_bert = []
for i in range(len(val_data["text"])):
    val_label = classifier(val_data["text"][i][:512])
    if val_label[0]["label"] == "battery":
        label_bert.append(1)
        score_bert.append(val_label[0]["score"])
    elif val_label[0]["label"] == "non-battery":
        label_bert.append(0)
        score_bert.append(val_label[0]["score"])

val_data["label_bert"] = label_bert
val_data["score_bert"] = score_bert

# result_path = r'C:\Users\Subways-Sun\OneDrive\Documents\GitHub\sodium-ion-batteries\data_annotated\annotated_data_openai_bert.json'
# with open(result_path, 'w', encoding='utf-8') as f:
#     json.dump(val_data, f, indent=4, ensure_ascii=False)



In [8]:
label_openai = val_data["label_openai"]
label_annotated = val_data["label_annotated"]
print(f"No\tAnno\tOpenAI\tBERT\tBERT Score")

openai_fp = 0
openai_fn = 0
bert_fp = 0
bert_fn = 0

for i in range(len(label_bert)):
    if (label_bert[i] + label_openai[i] + label_annotated[i] != 3) and (label_bert[i] + label_openai[i] + label_annotated[i] != 0):
        print(f"{i}\t{label_annotated[i]}\t{label_openai[i]}\t{label_bert[i]}\t{score_bert[i]}")
        if label_bert[i] == 1 and label_annotated[i] == 0:
            bert_fp += 1
        elif label_bert[i] == 0 and label_annotated[i] == 1:
            bert_fn += 1
        if label_openai[i] == 1 and label_annotated[i] == 0:
            openai_fp += 1
        elif label_openai[i] == 0 and label_annotated[i] == 1:
            openai_fn += 1

print(f"OpenAI False Positive: {openai_fp}")
print(f"OpenAI False Negative: {openai_fn}")
print(f"Bert False Positive: {bert_fp}")
print(f"Bert False Negative: {bert_fn}")

No	Anno	OpenAI	BERT	BERT Score
3	1	1	0	0.9992577433586121
7	0	1	0	0.9998243451118469
9	1	1	0	0.9997914433479309
17	1	1	0	0.9970126152038574
21	0	1	0	0.9998651742935181
26	1	1	0	0.9982491731643677
27	1	1	0	0.9997610449790955
34	0	1	0	0.9998968839645386
35	1	1	0	0.9983423948287964
36	0	1	0	0.9998403787612915
37	1	1	0	0.8129305839538574
39	1	1	0	0.9997953772544861
45	1	1	0	0.9991692304611206
47	1	1	0	0.9996803998947144
53	0	1	0	0.9998726844787598
72	1	1	0	0.9997242093086243
75	0	0	1	0.99140465259552
78	1	1	0	0.8448143601417542
79	0	1	0	0.9998418092727661
80	0	1	0	0.999821126461029
83	0	0	1	0.9931684732437134
86	1	1	0	0.9994577765464783
87	1	1	0	0.9997746348381042
89	1	1	0	0.9997076392173767
99	0	1	0	0.9998871088027954
OpenAI False Positive: 8
OpenAI False Negative: 0
Bert False Positive: 2
Bert False Negative: 15
